## Resource-Constrained Project Scheduling Problem with Transfer Times

This notebook demonstrates how to model and solve the Resource-Constrained Project Scheduling Problem with Sequence-Dependent Setup Times
using Constraint Programming with IBM’s CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API.
This problem extends the classical RCPSP (see [`rcpsp.ipynb`](https://github.com/radovluk/CP_Cookbook/blob/main/notebooks/rcpsp.ipynb)). [Problem defintion from Vilem Heinz](https://drive.google.com/file/d/1Gwfgm7mcY47d0zJWLY-SD9BVqHYsUKkP/view?usp=sharing)

### Problem Definiton

The Resource-Constrained Project Scheduling Problem with Trasfer Times (RCPSPTT) is an extension of the RCPSP problem defined. In RCPSP, activities are executed using available resources, precedence relationships between them must be adhered to, and the total makespan is minimized. Compared to the RCPSP, RCPSPTT introduces transfer times between activities. This means that resources do not have to be directly available after the activity that uses them finishes; rather, a transfer of resources to the following activity might be required. The transfer time depends on both the activities and the resource transferred.

Formally, the RCPSPTT problem is defined as follows:

Let $\mathcal{A}=\{0,1,\ldots,\hat{A}\}$, $\ \hat{A}\in\mathbb{N}$, be a set of activities with indices $i,j\in\mathcal{A}$.

Let $P\in\mathbb{N}^{|\mathcal{A}|}$ be a vector of activity processing times.

Let $\mathcal{E}=\{(i,j)\mid \text{activity } i \text{ precedes activity } j\}$ be a set of precedences.

Let $\mathcal{R}=\{0,1,\ldots,\hat{R}\}$, $\ \hat{R}\in\mathbb{N}$, be a set of primary resources indexed by $r\in\mathcal{R}$.

Let $C\in\mathbb{N}^{|\mathcal{R}|}$ be a vector of primary resource capacities.

Let $\mathbf{Q}\in\mathbb{Z}_{{\ge 0}}^{|\mathcal{A}|\times|\mathcal{R}|}$ be the activity–resource consumption matrix, with entries $\mathbf{Q}_{i,r}$ denoting the consumption of resource $r$ by activity $i$ ($i=0 \Rightarrow \mathbf{Q}_{i,r}=C_r$).

Let $\Delta\in\mathbb{Z}_{\ge 0}^{|\mathcal{A}|\times|\mathcal{A}|\times|\mathcal{R}|}$ be a matrix of transfer delays, with entries $\Delta_{i,j,r}$ denoting the delay required if resource $r$ is transferred between activities $i$ and $j$.

### CPLEX Formulation

$$
\begin{aligned}
\min \quad
& \operatorname{end}(a_{|\mathcal{A}|-1})
\qquad &\qquad & \text{(1)} \\[2mm]
\text{s.t.} \quad
& \operatorname{endBeforeStart}(a_i, a_j),
\qquad & \forall (i, j) \in \mathcal{E}
\quad & \text{(2)} \\[1mm]
& \sum_{j:(0, j, r) \in \mathcal{T}} f_{0, j, r} = C_r,
\qquad & \forall r \in \{0, \dots, |\mathcal{R}| - 1\}
\quad & \text{(3)} \\[1mm]
& f_{i,j,r} \ge 1 \implies \operatorname{presenceOf}(z_{i,j,r}) = 1,
\qquad & \forall (i, j, r) \in \mathcal{T}, \Delta_{i,j,r} = 0
\quad & \text{(4)?} \\[2mm]
& \sum_{\substack{j, k:(j, i, k) \in \mathcal{T}, \\ \Delta_{j, i, k} > 0, k=r}} \operatorname{heightAtStart}(z_{j, i, k}, \operatorname{pulse}(z_{j, i, k}, (0, U_{j, i, k}))) + \sum_{\substack{j, k:(j, i, k) \in \mathcal{T}, \\ \Delta_{j, i, k} = 0, k=r}} f_{j, i, k} = Q_{i, r},
\qquad & \forall i \in \{1, \dots, |\mathcal{A}| - 1\}, \forall r \in \{0, \dots, |\mathcal{R}| - 1\}
\quad & \text{(5)} \\[2mm]
& \sum_{\substack{j, k:(i, j, k) \in \mathcal{T}, \\ \Delta_{i, j, k} > 0, k=r}} \operatorname{heightAtStart}(z_{i, j, r}, \operatorname{pulse}(z_{i, j, r}, (0, U_{i, j, r}))) + \sum_{\substack{j, k:(i, j, k) \in \mathcal{T}, \\ \Delta_{i, j, k} = 0, k=r}} f_{i, j, k} = Q_{i, r},
\qquad & \forall i \in \{0, \dots, |\mathcal{A}| - 2\}, \forall r \in \{0, \dots, |\mathcal{R}| - 1\}
\quad & \text{(6)} \\[2mm]
& \operatorname{endBeforeStart}(a_i, z_{j, i, r}),
\qquad & \forall (i, j, r) \in \mathcal{T}
\quad & \text{(7)} \\[1mm]
& \operatorname{endBeforeStart}(z_{i, j, r}, a_j),
\qquad & \forall (i, j, r) \in \mathcal{T}
\quad & \text{(7)} \\[2mm]
& \sum_{i: \mathbf{Q}_{i, r} > 0} \operatorname{pulse}(a_i, \mathbf{Q}_{i, r}) + \sum_{\substack{(i, j, k) \in \mathcal{T}, \\ \Delta_{i, j, k} > 0, k=r}} \operatorname{pulse}(z_{i, j, r}, (0, U_{i, j, r})) \le C_r,
\qquad & \forall r \in \{0, \dots, |\mathcal{R}| - 1\}
\quad & \text{(8)} \\[2mm]
& a_i: \text{mandatory interval var, length } p_i,
\qquad & \forall i \in \mathcal{A}
\quad & \text{(9a)} \\[1mm]
& f_{i,j,r} \in \mathbb{Z}, f_{i,j,r} \in [0, U_{i,j,r}],
\qquad & \forall (i, j, r) \in \mathcal{T}, \Delta_{i,j,r}=0
\quad & \text{(9b)} \\[1mm]
& z_{i,j,r}: \text{optional interval var, length } \Delta_{i,j,r},
\qquad & \forall (i, j, r) \in \mathcal{T}, \Delta_{i,j,r}>0
\quad & \text{(9c)}
\end{aligned}
$$

**Objective:**
* (1) Minimize the makespan (the end time of the final sink activity $a_{|\mathcal{A}|-1}$).


**Modeling constraints:**
* (2) Enforces precedence relations $\mathcal{E}$ between activities, ensuring an activity must finish before its successor can start.
* (3) Ensures the total resource flow $r$ from the source node (activity 0) via instantaneous transfers ($f_{0,j,r}$) equals the total resource capacity $C_r$.
* (4) Implicates transfer activation for instantaneous transfers ($\Delta_{i,j,r} = 0$). If a flow $f_{i,j,r}$ exists, it implies the presence of the optional interval $z_{i,j,r}$. **Note: This constraint is contradictory as $f_{i,j,r}$ is defined for $\Delta=0$ and $z_{i,j,r}$ is defined for $\Delta>0$**.
* (5) Flow conservation (into activity): Ensures the total resource $r$ received by activity $i$ (summed from durative transfers $z$ and instantaneous flows $f$) equals its required quantity $Q_{i,r}$.
* (6) Flow conservation (out of activity): Ensures the total resource $r$ sent from activity $i$ (summed via $z$ and $f$) equals the quantity $Q_{i,r}$ that the activity processed (or $C_r$ for the source node $i=0$).
* (7) Temporal linking: Enforces the time delay for durative transfers ($z$). It ensures a transfer $z_{i,j,r}$ starts after $a_i$ ends, and $a_j$ starts after $z_{i,j,r}$ ends.
* (8) Resource capacity (cumulative constraint): Ensures that at any time, the total resource $r$ consumed by all active activities $a_i$ ($\mathbf{Q}_{i,r}$) and all active durative transfers $z$ does not exceed the capacity $C_r$.


**Variable Definitions:**
* (9a) $a_i$: A mandatory interval variable representing the execution of activity $i$ with length $p_i$.
* (9b) $f_{i,j,r}$: An integer variable representing the amount of resource flow $r$ between $i$ and $j$, defined only for instantaneous transfers ($\Delta_{i,j,r}=0$).
* (9c) $z_{i,j,r}$: An optional interval variable representing the transfer of resource $r$ between $i$ and $j$, with length $\Delta_{i,j,r}$. Defined only for durative transfers ($\Delta_{i,j,r}>0$).


**Parameters (Input Data):**
* $\mathcal{A}$: The set of activities.
* $\mathcal{E}$: The set of precedence relations (pairs of activities).
* $\mathcal{R}$: The set of primary resources.
* $\mathcal{T}$: The set of potential transfers (triples of activity, activity, resource).
* $p_i$: The fixed processing time (duration) of activity $a_i$.
* $C_r$: The maximum capacity available for resource $r$ (vector of primary resource capacities).
* $\Delta_{i,j,r}$: The fixed duration (delay) for a transfer $(i, j, r)$.
* $U_{i,j,r}$: The upper bound (maximum flow/capacity) of the transfer $(i, j, r)$.
* $Q_{i,r}$: The quantity of resource $r$ required by/transferred into/out of activity $i$.

### CPLEX Implementation

#### Imports

In [1]:
# from docplex.cp.model import *
# import docplex.cp.utils_visu as visu
import re
from pathlib import Path
from docplex.cp.model import CpoModel

#### Reading the data file

In [2]:
def parse_rcpsp_psplib(filepath):
    """
    Parses a .sm file (PSPLIB format for RCPSP with transfer times)
    and returns a dictionary with the project data.
    """
    with open(filepath, 'r') as f:
        content = f.read()

    data = {}

    # --- Get n_jobs and n_resources ---
    # Find: jobs (incl. supersource/sink ):  32
    match = re.search(r'jobs \(incl\. supersource/sink \):\s*(\d+)', content)
    data['n_jobs'] = int(match.group(1)) if match else 0
    
    # Find:  - renewable                 :  4   R
    match = re.search(r' - renewable\s*:\s*(\d+)', content)
    data['n_resources'] = int(match.group(1)) if match else 0

    n_jobs = data['n_jobs']
    n_res = data['n_resources']

    # --- Get Precedence Relations ---
    data['precedence_arcs'] = []
    # Find the start of the section
    prec_start = content.find('PRECEDENCE RELATIONS:')
    # Find the end of the section (the next '***' line)
    prec_end = content.find('****************', prec_start)
    prec_section = content[prec_start:prec_end]
    
    # Skip header lines
    for line in prec_section.splitlines()[2:]:
        if not line.strip():
            continue
        parts = [int(p) for p in line.strip().split()]
        predecessor = parts[0]
        successors = parts[3:]
        for succ in successors:
            # Convert 1-based index from file to 0-based index
            data['precedence_arcs'].append((predecessor - 1, succ - 1))

    # --- Get Durations and Demands ---
    data['durations'] = []
    data['demands'] = []
    req_start = content.find('REQUESTS/DURATIONS:')
    req_end = content.find('****************', req_start)
    req_section = content[req_start:req_end]

    # Skip header, blank, and '---' lines
    for line in req_section.splitlines()[3:]:
        if not line.strip():
            continue
        parts = [int(p) for p in line.strip().split()]
        # parts[0] = jobnr, parts[1] = mode
        data['durations'].append(parts[2])
        data['demands'].append(parts[3:]) # The rest are resource demands

    # --- Get Resource Capacities ---
    cap_start = content.find('RESOURCEAVAILABILITIES:')
    cap_end = content.find('****************', cap_start)
    cap_section = content[cap_start:cap_end]
    
    # Data is on the 3rd line (index 2) after the header
    cap_line = cap_section.splitlines()[2]
    data['capacities'] = [int(p) for p in cap_line.strip().split()]

    # --- Get Transfer Times ---
    data['transfer_times'] = []
    current_pos = cap_end # Start search from end of last section

    for _ in range(n_res):
        tt_start = content.find('TRANSFERTIMES', current_pos)
        tt_end = content.find('****************', tt_start)
        tt_section = content[tt_start:tt_end]
        
        matrix = []
        # Skip header, jobnr-header, and '---' lines
        lines = tt_section.splitlines()[3:]
        
        # Read the n_jobs lines of the matrix
        for i in range(n_jobs):
            line = lines[i]
            parts = [int(p) for p in line.strip().split()]
            # parts[0] is the row jobnr, the matrix data starts at index 1
            matrix.append(parts[1:])
            
        data['transfer_times'].append(matrix)
        current_pos = tt_end # Set position for next search

    return data

In [3]:
filename = "../data/rcpsptt/j301_a.sm"
filepath = Path(filename)
data = parse_rcpsp_psplib(filename)

A_hat = data['n_jobs'] - 1
R_hat = data['n_resources'] - 1

A_set = range(A_hat + 1) # Set of activities {0, ..., A_hat}
R_set = range(R_hat + 1) # Set of resources {0, ..., R_hat}

P = data['durations']                           # P[i]
C = data['capacities']                          # C[r]
Q = data['demands']                             # Q[i][r]
E = data['precedence_arcs']                     # List of (i, j) tuples
Delta_matrices = data['transfer_times']         # [r][i][j]

# For easier lookup, create a (i, j, r) -> delay dictionary
Delta = {
    (i, j, r): Delta_matrices[r][i][j]
    for i in A_set
    for j in A_set
    for r in R_set
}

# the upper bound U_i,j,r for any transfer is the total capacity C[r] of that resource
U = {
    (i, j, r): C[r]
    for i in A_set
    for j in A_set
    for r in R_set
    if i != j
}

# Build the T set (all potential transfers)
T = [(i, j, r) for i in A_set for j in A_set for r in R_set if i != j]

In [4]:
# ---
# (Assuming your parsing code and data prep is run here)
# ...
# A_set, R_set, P, C, Q, E, Delta, U, T are all defined
# ...
# ---

mdl = CpoModel()

# --- (9) Variable Definitions ---

# (9a) Activity interval variables
a = {i: mdl.interval_var(name=f"a_{i}", size=P[i]) for i in A_set}

# (9b) Optional transfer intervals z_(i,j,r) for Delta > 0
z = {
    (i, j, r): mdl.interval_var(size=Delta[i, j, r], optional=True, name=f'z_{i}_{j}_{r}')
    for (i, j, r) in T
    if Delta[i, j, r] > 0
}

# (9c) Flow amount f_(i,j,r) for instantaneous transfers (Delta == 0)
f = {
    (i, j, r): mdl.integer_var(min=0, max=U[i, j, r], name=f'f_{i}_{j}_{r}')
    for (i, j, r) in T
    if Delta[i, j, r] == 0
}

# (9d) **NEW:** Create a dictionary of pulse functions for all z intervals.
# This is the "function" that height_at_start was missing.
z_pulses = {
    (i, j, r): mdl.pulse(z[i, j, r], (0, U[i, j, r]))
    for (i, j, r) in z
}

# (9e) **FIX 1:** Link presence using the 2-argument height_at_start
for (i, j, r) in z:
    mdl.add(mdl.presence_of(z[i, j, r]) == (mdl.height_at_start(z[i, j, r], z_pulses[i, j, r]) > 0))


# --- (1) Objective Function ---
mdl.add(mdl.minimize(mdl.end_of(a[A_hat])))

# --- (2) Precedence constraint ---
mdl.add([mdl.end_before_start(a[i], a[j]) for (i, j) in E])

# --- (3) Source capacity release ---
# **FIX 2:** Use the 2-argument height_at_start
for r in R_set:
    outflow_instant = mdl.sum(
        f[0, j, r]
        for j in A_set
        if (0, j, r) in f
    )
    outflow_durative = mdl.sum(
        mdl.height_at_start(z[0, j, r], z_pulses[0, j, r])
        for j in A_set
        if (0, j, r) in z # Check if this durative transfer exists
    )
    mdl.add(outflow_instant + outflow_durative == C[r])

# --- (4) Transfer activation ---
# (Removed)

# --- (5) Flow conservation INTO activity i ---
# **FIX 3:** Use the 2-argument height_at_start
for i in range(1, A_hat + 1): # All activities except source
    for r in R_set:
        inflow_durative = mdl.sum(
            mdl.height_at_start(z[j, i, r], z_pulses[j, i, r])
            for j in A_set
            if (j, i, r) in z
        )
        inflow_instant = mdl.sum(
            f[j, i, r]
            for j in A_set
            if (j, i, r) in f
        )
        mdl.add(inflow_durative + inflow_instant == Q[i][r])

# --- (6) Flow conservation OUT of activity i ---
# **FIX 4:** Use the 2-argument height_at_start
for i in range(0, A_hat): # All activities except sink
    for r in R_set:
        outflow_durative = mdl.sum(
            mdl.height_at_start(z[i, j, r], z_pulses[i, j, r])
            for j in A_set
            if (i, j, r) in z
        )
        outflow_instant = mdl.sum(
            f[i, j, r]
            for j in A_set
            if (i, j, r) in f
        )
        mdl.add(outflow_durative + outflow_instant == Q[i][r])

# --- (7) Temporal linking ---
# (This was correct)
for (i, j, r) in z:
    mdl.add(mdl.end_before_start(a[i], z[i, j, r]))
    mdl.add(mdl.end_before_start(z[i, j, r], a[j]))

# --- (8) Resource capacity ---
# **FIX 5:** Sum the pre-defined pulses from the z_pulses dictionary
for r in R_set:
    # Activities consuming resource r
    activities_consuming = mdl.sum(
        mdl.pulse(a[i], Q[i][r])
        for i in A_set
        if Q[i][r] > 0
    )
    # Durative transfers consuming resource r
    transfers_consuming = mdl.sum(
        z_pulses[i, j, k] # Sum the pulse expressions directly
        for (i, j, k) in z
        if k == r
    )
    mdl.add(activities_consuming + transfers_consuming <= C[r])


# --- Solve ---
print("Solving model...")
ms = mdl.solve(log_output=True)
if ms:
    print("Solution found:")
    ms.print_solution()
else:
    print("No solution found.")

print("Model built successfully.")

Solving model...
 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 3996 variables, 10366 constraints
 ! Initial process time : 0.21s (0.21s extraction + 0.00s propagation)
 !  . Log search space  : 53050.5 (before), 53050.5 (after)
 !  . Memory usage      : 7.4 MB (before), 7.4 MB (after)
 ! Using parallel search with 12 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0       3996                 -
 + New bound is 0
 ! ----------------------------------------------------------------------------
 ! Search completed, model has no solution.
 ! Best bound             : 0
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 0
 ! Number of fails        : 0
 ! Total memory usage     : 29.9 MB (27.7 MB CP Optimizer + 2.2 MB Concert)
 ! Time spent in solve   

#### Create model and variables

In [5]:
mdl = CpoModel()

# (9a) Activity interval variables
a = [mdl.interval_var(name=f"a_{i}", size=P[i]) for i in A_set]

# (9b) Optional transfer intervals z_(i,j,r) for Delta > 0
z = {
    (i, j, r): mdl.interval_var(size=Delta[i, j, r], optional=True, name=f'z_{i}_{j}_{r}')
    for (i, j, r) in T
    if Delta[i, j, r] > 0
}

# (9c) Flow vars f_(i,j,r) 
f = {
    (i, j, r): mdl.integer_var(min=0, max=U[i, j, r], name=f'f_{i}_{j}_{r}')
    for (i, j, r) in T
    if Delta[i, j, r] == 0
}

#### Add constraints and define objective

In [6]:
# (1) Minimize the makespan
mdl.add(mdl.minimize(mdl.end_of(a[A_hat])))

In [7]:
# (2) Precedence constraint
mdl.add([mdl.end_before_start(a[i], a[j]) for (i, j) in E])

# (3) Source capacity release
for r in R_set:
    outflow_from_source = mdl.sum(
        f[0, j, r]
        for j in A_set
        if (0, j, r) in f
    )
    mdl.add(outflow_from_source == C[r])

# (4) Transfer activation
for (i, j, r) in T:
    if Delta[i, j, r] == 0 and (i, j, r) in z:
        # f_{i,j,r} >= 1 implies presenceOf(z_{i,j,r}) = 1
        mdl.add(mdl.if_then(f[i, j, r] >= 1, mdl.presence_of(z[i, j, r]) == 1))


# (5) Flow conservation INTO activity i (for i in {1, ..., |A|-1})
for i in range(1, A_hat + 1):
    for r in R_set:
        # Durative transfers incoming
        inflow_durative = mdl.sum(
            mdl.height_at_start(z[j, i, k], mdl.pulse(z[j, i, k], (0, U[j, i, k])))
            for j in A_set
            for k in R_set
            if (j, i, k) in z and Delta[j, i, k] > 0 and k == r
        )
        # Instantaneous transfers incoming
        inflow_instant = mdl.sum(
            f[j, i, k]
            for j in A_set
            for k in R_set
            if (j, i, k) in f and Delta[j, i, k] == 0 and k == r
        )
        mdl.add(inflow_durative + inflow_instant == Q[i][r])

# # (6) Flow conservation (OUT)
for i in range(0, A_hat):
    for r in R_set:
        # Durative transfers outgoing
        outflow_durative = mdl.sum(
            mdl.height_at_start(z[i, j, r], mdl.pulse(z[i, j, r], (0, U[i, j, r])))
            for j in A_set
            if (i, j, r) in z and Delta[i, j, r] > 0
        )
        # Instantaneous transfers outgoing
        outflow_instant = mdl.sum(
            f[i, j, k]
            for j in A_set
            for k in R_set
            if (i, j, k) in f and Delta[i, j, k] == 0 and k == r
        )
        mdl.add(outflow_durative + outflow_instant == Q[i][r])

# # (7) Temporal linking
for (i, j, r) in T:
    if (i, j, r) in z:
        # Transfer starts after activity i ends
        mdl.add(mdl.end_before_start(a[i], z[i, j, r]))
        # Activity j starts after transfer ends
        mdl.add(mdl.end_before_start(z[i, j, r], a[j]))

# # (8) Resource capacity
for r in R_set:
    # Activities consuming resource r
    activities_consuming = mdl.sum(
        mdl.pulse(a[i], Q[i][r])
        for i in A_set
        if Q[i][r] > 0
    )
    # Durative transfers consuming resource r
    transfers_consuming = mdl.sum(
        mdl.pulse(z[i, j, k], (0, U[i, j, k]))
        for (i, j, k) in T
        if Delta[i, j, k] > 0 and k == r and (i, j, k) in z
    )
    mdl.add(activities_consuming + transfers_consuming <= C[r])

#### Solve the model

In [8]:
print('Solving model...')
res = mdl.solve()
print('Solution: ')
res.print_solution()

Solving model...
 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 3996 variables, 7012 constraints
 ! Initial process time : 0.19s (0.19s extraction + 0.00s propagation)
 !  . Log search space  : 75497.3 (before), 75497.3 (after)
 !  . Memory usage      : 13.8 MB (before), 13.8 MB (after)
 ! Using parallel search with 12 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0       3996                 -
 + New bound is 0
 ! ----------------------------------------------------------------------------
 ! Search completed, model has no solution.
 ! Best bound             : 0
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 0
 ! Number of fails        : 0
 ! Total memory usage     : 33.0 MB (29.9 MB CP Optimizer + 3.1 MB Concert)
 ! Time spent in solve  

## Alternative CP formulation -- resource r is only occupied during an activity's execution do not track flows

$$
\begin{aligned}
\min \quad 
& \mathrm{endOf}(x_{\hat{A}}) 
\qquad &\qquad & \text{(1)} \\[2mm]
\text{s.t.} \quad
& \mathrm{alternative}\!\left(x_i,\; [\,y_{i1}\,]\right), 
\qquad & \forall i\in \mathcal{A} 
\quad & \text{(2)} \\[1mm]
& \sum_{i\in\mathcal{A}}
  \mathrm{pulse}\!\left(y_{i1},\, Q_{ir}\right)
  \le C_r, 
\qquad & \forall r\in \mathcal{R} 
\quad & \text{(3)} \\[1mm]
& \mathrm{endBeforeStart}(x_i, x_j), 
\qquad & \forall (i,j)\in \mathcal{E} 
\quad & \text{(4)} \\[1mm]
& (\mathrm{endOf}(x_i) + \Delta_{ijr} \le \mathrm{startOf}(x_j)) \ \vee \
\qquad & \forall r\in \mathcal{R},\ \forall (i,j)\in \mathcal{I}_r
\quad & \text{(5)} \\
& (\mathrm{endOf}(x_j) + \Delta_{jir} \le \mathrm{startOf}(x_i)), 
& & 
\quad & \\[1mm]
& \text{interval } x_i, 
\qquad & \forall i\in\mathcal{A} 
\quad & \text{(6)} \\[1mm]
& \text{interval } y_{i1}\ \text{optional, size}=P_{i}, 
\qquad & \forall i\in\mathcal{A} 
\quad & \text{(7)}
\end{aligned}
$$

(where $\mathcal{I}_r = \{ (i,j) \in \mathcal{A} \times \mathcal{A} \mid i < j, Q_{ir} + Q_{jr} > C_r \}$)

### Objective:

* **(1)** Minimize the makespan, defined as the end time of the unique sink activity $x_{\hat{A}}$.

### Modeling constraints:

* **(2)** Ensures the mandatory activity interval $x_i$ is synchronized with its single corresponding (optional) mode interval $y_{i1}$.

* **(3)** Enforces renewable resource limits. At any time, the sum of demands from all executing activities ($\mathrm{pulse}(y_{i1}, ...)$) must not exceed the resource capacity $C_r$.

* **(4)** Maintains all technological precedence relations $(i,j)\in \mathcal{E}$.

* **(5)** Enforces transfer time disjunctions. For any resource $r$, and for any pair of activities $(i,j)$ in the "incompatible set" $\mathcal{I}_r$ (meaning their combined demand exceeds capacity), this constraint forces them to be scheduled sequentially, respecting the required transfer time.

### Variables:

* **(6)** $x_i$: a mandatory interval representing the execution of activity $i$.

* **(7)** $y_{i1}$: an optional interval, representing the single mode of activity $i$ with fixed duration $P_{i}$.

### Symbol and Function Reference

| **Symbol / Function** | **Meaning** | **docplex.cp reference** | 
| :--- | :--- | :--- |
| $\mathcal{A} = \{0..N\}$ | Set of activities (0-indexed) | — | 
| $\mathcal{R} = \{0..R\}$ | Set of renewable resources (0-indexed) | — | 
| $P_i$ | Processing time (duration) of activity $i$ | — | 
| $Q_{ir}$ | Renewable demand of activity $i$ on resource $r$ | — | 
| $C_r$ | Capacity of renewable resource $r$ | — | 
| $\mathcal{E}$ | Set of precedence relations $(i, j)$ | — | 
| $\Delta_{ijr}$ | Transfer time for resource $r$ from $i$ to $j$ | — | 
| $\mathcal{I}_r$ | Set of incompatible pairs for resource $r$ | — | 
| $x_i$ | Mandatory interval variable for activity $i$ | `interval_var` | 
| $y_{i1}$ | Optional interval for the single mode of activity $i$ | `interval_var` | 
| $\mathrm{alternative}(x_i, [y_{i1}])$ | Selects mode $y_{i1}$ and synchronizes it with $x_i$ | `alternative` | 
| $\mathrm{pulse}(y_{i1}, Q_{ir})$ | Time-varying usage of resource $r$ by activity $i$ | `pulse` | 
| $\mathrm{endBeforeStart}(x_i, x_j)$ | Enforces $\mathrm{endOf}(x_i) \le \mathrm{startOf}(x_j)$ | `end_before_start` | 
| $\mathrm{endOf}(x_i)$ | End time of interval $x_i$ | `end_of` | 
| $\mathrm{startOf}(x_i)$ | Start time of interval $x_i$ | `start_of` | 
| $\vee$ | Logical "OR" operator | `|` | 
| $\min \mathrm{endOf}(x_{\hat{A}})$ | Objective: minimize makespan | `minimize` |

#### Code

In [9]:
mdl = CpoModel()

# (6) x[i]: mandatory interval for activity i
x = [mdl.interval_var(name=f'x_{i+1}') for i in A_set]

# (7) y[i]: optional interval for the single mode of activity i
y = [mdl.interval_var(name=f'y_{i+1}_1', optional=True, size=P[i]) for i in A_set]

# (2) Link mandatory and optional variables 
[mdl.add(mdl.alternative(x[i], [y[i]])) for i in A_set]

# (3) Cumulative Resource Constraints 
[
    mdl.add(mdl.sum([mdl.pulse(y[i], Q[i][r]) for i in A_set if Q[i][r] > 0]) <= C[r])
    for r in R_set if any(Q[i][r] > 0 for i in A_set)
]

# (4) Technological Precedence Constraints
[mdl.add(mdl.end_before_start(x[i], x[j])) for (i, j) in E]

# (5) Transfer Time Constraints
[
    mdl.add(
        # Option 1: i -> j (with transfer delay)
        (mdl.end_of(x[i]) + Delta[(i, j, r)] <= mdl.start_of(x[j])) |
        # Option 2: j -> i (with transfer delay)
        (mdl.end_of(x[j]) + Delta[(j, i, r)] <= mdl.start_of(x[i]))
    )
    for r in R_set
    for i in A_set
    for j in A_set
    # All conditions to find an incompatible pair (i, j) on resource r:
    if i < j and Q[i][r] > 0 and Q[j][r] > 0 and (Q[i][r] + Q[j][r] > C[r])
]

# (1) Objective Function
mdl.add(mdl.minimize(mdl.end_of(x[A_hat])))

print('Solving model...')
res = mdl.solve()
print('Solution: ')
res.print_solution()

Solving model...
 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 64 variables, 130 constraints


 ! Initial process time : 0.02s (0.02s extraction + 0.00s propagation)
 !  . Log search space  : 160.0 (before), 160.0 (after)
 !  . Memory usage      : 696.6 kB (before), 696.6 kB (after)
 ! Using parallel search with 12 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         64                 -
 + New bound is 37
 ! Using iterative diving.
 *            68      128  0.12s        1      (gap is 45.59%)
 *            65      345  0.12s        1      (gap is 43.08%)
 *            62      472  0.12s        1      (gap is 40.32%)
 *            60      708  0.12s        1      (gap is 38.33%)
 *            57      953  0.12s        1      (gap is 35.09%)
              57     1000         12    1   F     5  = startOf(y_11_1)
              57     2000          6    1   F         !presenceOf(y_16_1)
              57     3000          6    1         4  = startOf(y

## Alternative CP formulation -- formulate problem like a flow -- search space too big

$$
\begin{aligned}
\min \quad 
& \mathrm{endOf}(x_{\hat{A}}) 
\qquad &\qquad & \text{(1)} \\[2mm]
\text{s.t.} \quad
& \mathrm{endBeforeStart}(x_i, x_j), 
\qquad & \forall (i,j)\in \mathcal{P} 
\quad & \text{(2)} \\[1mm]
& \mathrm{if\_then}\!\left(y_{ijk} = 1,\; \mathrm{endOf}(x_i) + \Delta_{ijk} \le \mathrm{startOf}(x_j)\right), 
\qquad & \forall i,j,k, i \neq j 
\quad & \text{(3)} \\[1mm]
& f_{ijk} \le Q_{ik} \cdot y_{ijk} \quad \text{and} \quad f_{ijk} \le Q_{jk} \cdot y_{ijk}, 
\qquad & \forall i,j,k, i \neq j 
\quad & \text{(4)} \\[1mm]
& \sum_{j \in \mathcal{A} \setminus \{i\}} f_{jik} = Q_{ik}, 
\qquad & \forall i\in \mathcal{A}, \forall k \in \mathcal{R} 
\quad & \text{(5)} \\[1mm]
& \sum_{j \in \mathcal{A} \setminus \{i\}} f_{ijk} = Q_{ik}, 
\qquad & \forall i\in \mathcal{A}, \forall k \in \mathcal{R} 
\quad & \text{(6)} \\[2mm]
& \text{interval } x_i,\ \text{size} = P_{i}, 
\qquad & \forall i\in\mathcal{A} 
\quad & \text{(7)} \\[1mm]
& \text{binary } y_{ijk}, 
\qquad & \forall i,j,k, i \neq j 
\quad & \text{(8)} \\[1mm]
& \text{integer } f_{ijk} \ge 0, 
\qquad & \forall i,j,k, i \neq j 
\quad & \text{(9)}
\end{aligned}
$$

**Objective:**
- **(1)** Minimize the makespan, defined as the end time of the unique sink activity $x_{\hat{A}}$.

**Modeling constraints**:
- **(2)** Maintains all technological precedence relations $(i,j)\in \mathcal{P}$.
- **(3)** Enforces transfer times. IF a resource $k$ is transferred from $i$ to $j$ ($y_{ijk}=1$), THEN $j$ must start after $i$ finishes, respecting the transfer time $\Delta_{ijk}$.
- **(4)** Links flow $f$ to decision $y$. A flow $f_{ijk}$ can only be non-zero if the decision to transfer $y_{ijk}$ is 1. The flow is also capped by the sender's ($Q_{ik}$) and receiver's ($Q_{jk}$) resource requirement.
- **(5)** Flow conservation (inflow). Each activity $i$ must *receive* (from all other activities $j$) exactly its required amount $Q_{ik}$ of each resource $k$.
- **(6)** Flow conservation (outflow). Each activity $i$ must *send* (to all other activities $j$) exactly its required amount $Q_{ik}$ of each resource $k$.

**Variables**:
- **(7)** $x_i$: A mandatory interval variable (`interval_var`) representing the execution of activity $i$ with a fixed processing time $P_{i}$.
- **(8)** $y_{ijk}$: A binary variable (`binary_var`), which is 1 if at least one unit of resource $k$ is transferred from $i$ to $j$, and 0 otherwise.
- **(9)** $f_{ijk}$: An integer variable (`integer_var`) representing the exact number of units of resource $k$ transferred from $i$ to $j$.

---

### Symbol and Function Reference

| **Symbol / Function** | **Meaning** | **docplex.cp reference** | 
| :--- | :--- | :--- |
| $\mathcal{A} = \{0..\hat{A}\}$ | Set of activities (incl. dummy start 0 and end $\hat{A}$) | — | 
| $\mathcal{R} = \{0..\hat{R}\}$ | Set of renewable resources | — | 
| $\mathcal{P}$ | Set of precedence relations $(i, j)$ | — | 
| $P_i$ | Processing time (duration) of activity $i$ | — | 
| $Q_{ik}$ | Resource demand of activity $i$ for resource $k$ | — | 
| $\Delta_{ijk}$ | Transfer time for resource $k$ from $i$ to $j$ | — | 
| $x_i$ | Mandatory interval variable for activity $i$ | `interval_var` | 
| $y_{ijk}$ | Binary variable (transfer decision) | `binary_var` | 
| $f_{ijk}$ | Integer variable (flow amount) | `integer_var` | 
| $\mathrm{presenceOf}(x_i)$ | 1 if interval $x_i$ is present, 0 otherwise | `presence_of` |
| $\mathrm{endBeforeStart}(x_i, x_j)$ | Enforces $\mathrm{endOf}(x_i) \le \mathrm{startOf}(x_j)$ | `end_before_start` | 
| $\mathrm{if\_then}(y, \text{constr})$ | Logical "if-then" constraint | `if_then` | 
| $\mathrm{endOf}(x_i)$ | End time of interval $x_i$ | `end_of` | 
| $\mathrm{startOf}(x_i)$ | Start time of interval $x_i$ | `start_of` | 
| $\min \mathrm{endOf}(x_{\hat{A}})$ | Objective: minimize makespan | `minimize` |

In [10]:
mdl = CpoModel(name="RCPSPTT_FlowBased")

# (7) interval variables (task execution, mandatory size P[i])
x = {i: mdl.interval_var(name=f"x_{i}", size=P[i]) for i in A_set}

# (8) binary variables (transfer decision: 1 if resource k moves from i to j)
y = {(i, j, k): mdl.binary_var(name=f"y_{i}_{j}_{k}")
     for i in A_set for j in A_set for k in R_set if i != j}

# (9) integer variables (flow amount: units of k moving from i to j)
f = {(i, j, k): mdl.integer_var(0, C[k], name=f"f_{i}_{j}_{k}")
     for i in A_set for j in A_set for k in R_set if i != j}

# (1) objective
mdl.add(mdl.minimize(mdl.end_of(x[A_hat])))

# dummy start activity begins at time 0
mdl.add(mdl.start_of(x[0]) == 0)

# (2) precedence constraints
mdl.add([mdl.end_before_start(x[i], x[j]) for (i,j) in E])

for k in R_set:
    for i in A_set:
        # (5) flow Conservation (inflow): total received resources must meet demand Q[i][k].
        mdl.add(mdl.sum(f.get((j, i, k), 0) for j in A_set if j != i) == Q[i][k])
        # (6) flow Conservation (outflow): total sent resources must equal demand Q[i][k].
        mdl.add(mdl.sum(f.get((i, j, k), 0) for j in A_set if j != i) == Q[i][k])

        for j in A_set:
            if i == j: continue
            # (3) transfer time constraint: if y[(i,j,k)] = 1, apply time delay Delta.
            mdl.add(mdl.if_then(y[(i, j, k)] == 1, mdl.end_of(x[i]) + Delta.get((i, j, k), 0) <= mdl.start_of(x[j])))
            # (4) flow f is capped by sender's Q[i][k] and y[(i,j,k)].
            mdl.add(f[(i, j, k)] <= Q[i][k] * y[(i, j, k)])
            # (4) flow f is capped by receiver's Q[j][k] and y[(i,j,k)].
            mdl.add(f[(i, j, k)] <= Q[j][k] * y[(i, j, k)])

print('Solving model...')
res = mdl.solve(FailLimit=100000,TimeLimit=10, log_output=True)
print('Solution: ')
res.print_solution()

Solving model...
 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 7968 variables, 12209 constraints
 ! Presolve      : 74 extractables eliminated
 ! FailLimit            = 100000
 ! TimeLimit            = 10
 ! Initial process time : 0.18s (0.18s extraction + 0.00s propagation)
 !  . Log search space  : 4398.7 (before), 4398.7 (after)
 !  . Memory usage      : 5.9 MB (before), 5.9 MB (after)
 ! Using parallel search with 12 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0       7968                 -
 + New bound is 37
 ! Using temporal relaxation.
                     1000       3984    1   F    80 <= startOf(x_1)
                     2000       3984    1        13 >= startOf(x_1)
                     1000       3984    2   F     9 <= startOf(x_1)
                     2000       3984    2       182 >=

## Aditional Resources

- **Instances for RCPSPTT**
  - https://www2.informatik.uni-osnabrueck.de/kombopt/data/rcpsp/

- **For image of the problem instance see this article:** [An efficient genetic algorithm to solve the resource-constrained project scheduling problem with transfer times](https://www.sciencedirect.com/science/article/pii/S0377221717306549)

- [Problem defintion](https://drive.google.com/file/d/1Gwfgm7mcY47d0zJWLY-SD9BVqHYsUKkP/view?usp=sharing)